# OpenAI Gym Cartpole-v1 Beispiel

Dieses Beispiel basiert auf dem offiziellen OpenAI Gym [Tutorial](https://gym.openai.com/docs/)

In [1]:
import gym  # If gym isn't installed yet in your environment, then you can install it via pip: pip install gym
import numpy as np

Führt die nächste Zelle aus, um zu testen, dass alles funktioniert.
Es sollte ein Fenster erscheinen, das für ein paar Sekunden das CartPole
Environment zeigt.

In [2]:
env = gym.make('CartPole-v1')
env.reset()
for _ in range(200):
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
env.close()

c:\users\yves\gym\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Environments

Die Hauptkomponenten von OpenAI Gym sind die verschiedenen Environments. Die Funktion
```gym.make(environment_name)``` erstellt ein neues Environment.




In [3]:
env = gym.make('CartPole-v1')

Jedes Environment hat einen Observation-Space und einen Action-Space. Der Observation-Space beschreibt, welche Arten von Observationen euer Agent machen kann.

In [4]:
env.observation_space

Box(4,)

```Box(4,)``` heisst, dass alle Observationen, die euerem System zu Verfügung stehen 4-dimensional sind. Den Bereich der Dimensionen kann mit ```env.observation_space.low``` und
```env.observation_space.high``` eingesehen werden.

In [5]:
env.observation_space.low

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

In [6]:
env.observation_space.high

array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
      dtype=float32)

Im Fall des Cartpole-Environments stehen die 4 Dimensionen für:
* Dim 0: Position des Wagens
* Dim 1: Geschwindigkeit des Wagens
* Dim 2: Winkel des Pendels
* Dim 3: Winkelgeschwindigkeit des Pendels

Der Action-Space beschreibt, welche Aktionen ein Agent im Environment vornehmen kann. Das heisst, welchen Input ```env.step(action)``` erwartet.

In [7]:
env.action_space

Discrete(2)

Der CartPole Action-Space ist ```Discrete(2)```. 
```Discrete(n)``` heisst, das Environment erwartet eine von n discreten Aktionen als Input erwarted: ```0 <= action < n```.

Im Fall des CartPole Environments wird mit 0 und 1 bestimmt, ob der Wagen nach links oder rechts verschoben wird.

## Wichtigste Funktionen

* ```env = gym.make('CartPole-v1')``` erstellt ein neues CartPole Environment
* ```observation = env.reset()``` initialisiert das Environment in einen neuen Startzustand und gibt die erste Observation für euren Agenten zurück.
* ```observation, reward, done, info = env.step(action)``` Update Funktion des Environments. Gibt neue Observation und die Belohnung zurück. Wenn ```done``` auf ```True``` gesetzt ist, befindet sich das Environment in einem Endzustand und sollte mit ```env.reset()``` zurückgesetzt werden. ```info``` enthält zusätzliche Informationen, sollte für CartPole leer sein.
* ```env.render()``` zeigt eine graphische Representation des Zustands des Environments an.
* ```env.close()``` beendet das Environment. Je nach Konfiguration eures Computers, muss dies nach ```env.render()``` aufgerufen werden, damit sich das Fenster schliesst.

## Modell

In dieser Einführung verwenden wir ein sehr simples Kontroller-Modell und optimisieren es mit  2 verschiedenen Random-Search Varianten.

Als Modellparameter verwenden wir $\theta \in \mathcal{R}^4$ und die nächste Aktion wählen wir gemäss Vorzeichen des Skalarprodukts zwischen Parametervektor und Beobachtungsvektor:
\begin{equation*}
    action =
    \begin{cases}
        0 & \theta^T x_{obs} < 0 \\
        1 & else
    \end{cases}
\end{equation*}

Simpler Training Loop für naiven Random Search

In [8]:
env = gym.make('CartPole-v1')
best_param = np.zeros(4)
best_cumulative_reward = 0.0
for epoch in range(2000):
    observation = env.reset()
    param = 2 * np.random.rand(4) - 1 # initialize new random parameter
    done = False
    cumulative_reward = 0.0
    while not done:
        # choose action according to our model
        action = 0 if np.dot(param, observation) < 0.0 else 1
        observation, reward, done, _ = env.step(action)
        cumulative_reward += reward
    
    if cumulative_reward > best_cumulative_reward:
        best_param = param
        best_cumulative_reward = cumulative_reward

Visualisierung des gelernten Verhaltens:

In [9]:
env = gym.make('CartPole-v1')
done = False
observation = env.reset()
cum_reward = 0.0
while not done:
    env.render()
    action = 0 if np.dot(best_param, observation) < 0.0 else 1
    observation, reward, done, _  = env.step(action)
    cum_reward += reward
env.close()
print(cum_reward)

500.0


## Hill-Climbing

Das folgende Beispiel wurde von [hier](http://kvfrans.com/simple-algoritms-for-solving-cartpole/) übernommen. Anstatt nach jeder Iteration den Parametervektor komplett neu zu wählen addieren wir eine kleine zufällige Veränderung zum besten bisherigen Parameter.

Wir haben die Logik für unseren Agenten in eine separate Klasse verschoben.

In [10]:
class HillClimbRandomPolicy(object):
    def __init__(self, dim=4):
        self.dim = dim
        self.param = self._resample()
        self.max_reward = 0.0
        self.epsilon = 0.1
        self.best_param = self.param
        
    def _resample(self):
        return 2 * np.random.rand(self.dim) - 1
    
    def action(self, observation):
        return 0 if np.dot(observation, self.param) < 0.0 else 1
    
    def best_action(self, observation):
        return 0 if np.dot(observation, self.best_param) < 0.0 else 1
    
    def update(self, history):
        total_reward = np.sum([h['reward'] for h in history])
        if total_reward > self.max_reward:
            self.max_reward = total_reward
            self.best_param = self.param
        else:
            self.param = self.best_param
        self.param += self.epsilon * self._resample()

Der folgende Trainingloop ist sehr generisch. Obwohl nicht strikt notwendig für unseren simplen Agenten, sammeln wir den kompletten Verlauf aller Aktionen, Observationen und Belohnungen, da viele cleverere Algorithmen diese Information brauchen.

In [11]:
env = gym.make('CartPole-v1')
policy = HillClimbRandomPolicy()
for epoch in range(30000):
    done = False
    observation = env.reset()
    history = []
    while not done:
        action = policy.action(observation)
        history_elem = {}
        history_elem['observation'] = observation
        history_elem['action'] = action
        observation, reward, done, _ = env.step(action)
        history_elem['reward'] = reward
        
        history.append(history_elem)

    policy.update(history)

Visualisiert das gelernte verhalten.

In [12]:
env = gym.make('CartPole-v1')
done = False
observation = env.reset()
cum_reward = 0.0
while not done:
    env.render()
    action = policy.best_action(observation)
    observation, reward, done, _  = env.step(action)
    cum_reward += reward
env.close()
print(cum_reward)

10.0


## Ende

Das CartPole Environment gilt als gelöst, wenn der Agent in 100 konsekutiven Runs eine durchschnittliche Belohnung von 200 erreicht.
Viel Spass!